In [44]:
import os
import pandas as pd
import numpy as np
import json
from collections import defaultdict

cwd = os.getcwd()

## REVIEWS DATASET CLEANUP

In [20]:
MIN_USER_REVIEWS = 6 

user_review_counts = defaultdict(int)

chunk_size = 25000 
reviews_path = cwd+"/yelp_academic_dataset_review.json"

# Conteo de reviews por usuario
for chunk in pd.read_json(reviews_path, lines=True, chunksize=chunk_size):
    # Se hace por chunks
    user_counts_chunk = chunk["user_id"].value_counts()
    for user_id, count in user_counts_chunk.items():
        user_review_counts[user_id] += count

user_counts_df = pd.DataFrame.from_dict(
    user_review_counts, orient="index", columns=["review_count"]
).reset_index(names=["user_id"])

In [22]:
# Condicional: Solo usuarios con 6 o más reviews
valid_users = user_counts_df[
    user_counts_df["review_count"] >= MIN_USER_REVIEWS
]["user_id"].tolist()

pd.Series(valid_users).to_csv("valid_users.csv", index=False)

In [36]:
len(valid_users)

228195

In [28]:
output_path = "filtered_reviewsv3.json"

# Columnas a eliminar
columns_to_drop = ["useful", "funny", "cool", "text", "date"]

with open(output_path, "w") as f_out:
    for chunk in pd.read_json(reviews_path, lines=True, chunksize=chunk_size):
        filtered_chunk = chunk[chunk["user_id"].isin(valid_users)]
        filtered_chunk = filtered_chunk.drop(columns=columns_to_drop, errors="ignore")
        if not filtered_chunk.empty:
            filtered_chunk.to_json(f_out, orient="records", lines=True, mode="a")

## Con el dataset de reviews limpio, podemos proceder a limpiar el dataset de usuarios y lugares.

In [34]:
# Paths
filtered_reviews_path = cwd+"/filtered_reviewsv3.json"
users_path = "yelp_academic_dataset_user.json"          
businesses_path = "yelp_academic_dataset_business.json"

# Debido a que son muchos registros, los Ids a guardar se guardan en una lista y se lee el dataset por chunks
valid_user_ids = set()
valid_business_ids = set()

chunk_size = 100_000

for chunk in pd.read_json(filtered_reviews_path, lines=True, chunksize=chunk_size):
    valid_user_ids.update(chunk["user_id"].unique())
    valid_business_ids.update(chunk["business_id"].unique())

print(f"Unique users: {len(valid_user_ids)}, Unique businesses: {len(valid_business_ids)}")

Unique users: 228195, Unique businesses: 147861


In [38]:
cleaned_users_path = "cleaned_users.json"

#Columnas a eliminar de Usuarios
columns_to_drop_users = ["yelping_since", "useful", "funny", "cool", "elite", "fans", "compliment_more", "compliment_profile", "compliment_cute", "compliment_list", "compliment_note", "compliment_plain", "compliment_cool", "compliment_funny", "compliment_writer", "compliment_photos"]


with open(cleaned_users_path, "w") as f_out:
    for chunk in pd.read_json(users_path, lines=True, chunksize=chunk_size):
        filtered_users = chunk[chunk["user_id"].isin(valid_user_ids)]
        filtered_users = filtered_users.drop(columns=columns_to_drop_users, errors="ignore")
        if not filtered_users.empty:
            filtered_users.to_json(f_out, orient="records", lines=True, mode="a")

In [48]:
cleaned_businesses_path = "cleaned_businesses.json"

columns_to_drop_business = ["city","postal_code","latitude","longitude","is_open","address", "attributes","hours"]

# Filtrado del dataset de Negocios
with open(cleaned_businesses_path, "w") as f_out:
    for chunk in pd.read_json(businesses_path, lines=True, chunksize=chunk_size):
        filtered_businesses = chunk[chunk["business_id"].isin(valid_business_ids)]
        filtered_businesses = filtered_businesses.drop(columns=columns_to_drop_business, errors="ignore")
        if not filtered_businesses.empty:
            filtered_businesses.to_json(f_out, orient="records", lines=True, mode="a")